In [18]:
import os
import re
import sys

import pandas as pd
from tqdm.auto import tqdm

In [11]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [12]:
output_dir = '../data'
output_path = os.path.join(output_dir, 'df_cleaned.parquet')

try:
    df = pd.read_parquet(output_path)
except FileNotFoundError:
    print("Файл отсутствует. Проверьте этап запуска.")

In [13]:
from src.keyphrase_extraction import extract_yake_keyphrases
from src.search import TfidfSearch
from src.preprocessing import preprocess_text

pd.set_option('display.max_colwidth', 300)
tqdm.pandas()

## 1. Извлечение ключевых фраз: YAKE

In [14]:
print("Извлечение ключевых фраз с помощью YAKE...")
df['yake_keywords'] = df['abstract'].progress_apply(extract_yake_keyphrases)
print("Извлечение завершено.")

Извлечение ключевых фраз с помощью YAKE...


  0%|          | 0/19276 [00:00<?, ?it/s]

Извлечение завершено.


In [15]:
print("Примеры извлеченных ключевых фраз:")
display(df[['abstract', 'yake_keywords']].head())

Примеры извлеченных ключевых фраз:


,abstract,yake_keywords
0,"В статье рассматривается сущность и диагностика морально-психологического состояния личного состава. Отмечается, что морально-психологическое состояние исторически устойчивый термин. Возникновение его в начале 1990-х гг. подтвердилось в настоящее время во всех современных армиях, ему придается о...","[состояния личного состава, статье рассматривается сущность, диагностика морально-психологического состояния, морально-психологического состояния личного, личного состава, статье рассматривается, рассматривается сущность]"
1,"Вводятся в научный оборот результаты палеокарпологического анализа керамики поселения черкаскульской культуры бронзового века Ольховка, расположенного в долине Исети. Обнаружение на черепках отпечатков семян пшеницы и ячменя практически первое прямое свидетельство знакомства черкаскульского насе...","[бронзового века Ольховка, научный оборот результаты, оборот результаты палеокарпологического, результаты палеокарпологического анализа, палеокарпологического анализа керамики, анализа керамики поселения, керамики поселения черкаскульской]"
2,"Статья посвящена одонтологической характеристике населения ямной культурно-исторической общности. На основании одонтологических данных проанализирован состав ямного населения Калмыкии, Астраханской области и Украины. Сделан вывод о морфологической неоднородности ямников и ведущей роли в формиров...","[Статья посвящена одонтологической, ямной культурно-исторической общности, посвящена одонтологической характеристике, одонтологической характеристике населения, характеристике населения ямной, населения ямной культурно-исторической, Статья посвящена]"
3,"В статье рассматривается содержание писем и рассказов фронтовиков, оказывавшее разноплановое влияние на восприятие горожанами поволжских губерний действительности в годы Первой мировой войны. Отмечено его содействие развитию у населения чувства патриотизма. Также на конкретных примерах показано ...","[Первой мировой войны, годы Первой мировой, статье рассматривается содержание, восприятие горожанами поволжских, горожанами поволжских губерний, поволжских губерний действительности, оказывавшее разноплановое влияние]"
4,"В статье представлен анализ системы управления Московских приказов и городов в Смутное время начала XVII в. Реконструкция 700 биографий служащих (дьяков и подьячих) позволила выявить основные признаки бюрократии и показать, что кризис верховной власти не приводит к разрушению сложившихся институ...","[системы управления Московских, управления Московских приказов, статье представлен анализ, представлен анализ системы, анализ системы управления, Московских приказов, городов в Смутное]"


YAKE используется как базовый, статистический подход. Он позволяет быстро получить ключевые фразы, однако имеет ряд системных недостатков
Например:
Алгоритм слепо извлекает последовательности слов, что часто приводит к обрывкам фраз, которые не являются самостоятельными терминами.
*   `морально-психологического состояния личного` (Пример 0) — чего "личного"? Фраза обрывается.
*   `системы управления Московских` (Пример 4) — чего "Московских"? Приказов. Фраза неполная.
*   `керамики поселения черкаскульской` (Пример 1) — чего "черкаскульской"? Культуры.

Для преодоления этих проблем, таких как отсутствие семантического понимания и генерация шаблонных фраз, далее будет применен подход на основе большой языковой модели, способной к обобщению и пониманию контекста

## 2. Поиск по смыслу: TF-IDF

In [24]:
from src.preprocessing import lemmatize_word 

In [16]:
tfidf_search_engine = TfidfSearch()

tfidf_search_engine.build_index(
    lemmatized_texts=df['lemmatized_abstract'],
    original_texts=df['abstract']
)

Building TF-IDF index...
TF-IDF index built successfully.


In [28]:
def highlight_keywords(text: str, query: str, preprocessor_func) -> str:
    """
    Выделяет в оригинальном тексте слова, леммы которых присутствуют в запросе.
    """
    query_lemmas = set(preprocessor_func(query).split())
    
    tokens = re.split(r'(\s+)', text) 
    
    highlighted_tokens = []
    for token in tokens:
        if not token.strip():
            highlighted_tokens.append(token)
            continue
        
        cleaned_token = token.strip('.,!?:;()[]{}')
        
        if lemmatize_word(cleaned_token.lower()) in query_lemmas:
            highlighted_tokens.append(f"**{token}**")
        else:
            highlighted_tokens.append(token)
            
    return "".join(highlighted_tokens)

In [29]:
from IPython.display import display, Markdown

In [30]:
queries = [
    "статьи про графовые алгоритмы в компьютерном зрении",
    "модернизация производственных процессов на предприятии",
    "анализ социальных сетей и выявление сообществ"
]

for query in queries:
    print(f"--- Результаты поиска для запроса: '{query}' ---")
    
    search_results = tfidf_search_engine.search(query, preprocessor_func=preprocess_text)
    
    if not search_results:
        print("Ничего не найдено.")
    else:
        for i, (doc_index, doc_text) in enumerate(search_results):
            highlighted = highlight_keywords(doc_text, query, preprocess_text)
            md_output = f"""{i+1}. (Индекс документа: {doc_index})
            > {highlighted[:500]}..."""
            display(Markdown(md_output))

--- Результаты поиска для запроса: 'статьи про графовые алгоритмы в компьютерном зрении' ---


1. (Индекс документа: 14032)
            > В работе предлагается **компьютерный** метод получения упрощенной макромодели динамической системы, который базируется на методах выделения доминантных собственных значений системы. Представлен **алгоритм** **компьютерного** аппроксимационного преобразования сложной модели к более простому видуI...

2. (Индекс документа: 13067)
            > Рассматриваются **алгоритмы** 3D-реконструкции и их применимость для решения задач **компьютерного** **зрения** на процессорах с низким энергопотреблением. Проведенный анализ быстродействия и вычислительной сложности **алгоритмов** позволяет определить их преимущества и сделать оптимальный выбор в зависимости от условий их реализации....

3. (Индекс документа: 14135)
            > В **статье** сформированы методы обнаружения лиц и радужной оболочки глаза в системах **компьютерного** **зрения** на основе методов AdaBoost и локальных бинарных шаблонов, а также использование цветных пространств для фильтрации изображения...

4. (Индекс документа: 6715)
            > Проблемы передачи пространственных отношений в различных языках уже давно стали предметом изучения в лингвистике. Различные лингвистические школы предпринимали попытки описать и классифицировать их. Однако нерешенным пока остается целый комплекс проблем, связанный с **компьютерной** лингвистикой. В данной **статье** автор делает попытку разработать классификацию, которая была бы применима в **алгоритме** **компьютерного** перевода с английского языка на турецкий. В настоящее время этот **алгорит...

...

1. (Индекс документа: 13136)
            > Представлен обзор работ, посвященных проблеме **анализа** компьютерных **социальных** **сетей.** Существует четыре основных направления исследований в данной области: структурное, ресурсное, нормативное и динамическое. Для решения различных задач при **анализе** **социальных** **сетей** используются графовые и стохастические модели, модели эволюции **сетей,** методы с привлечением онтологий, структурные и реляционные модели, методы машинного обучения, методы визуализации графов и т. д. Приведено...

2. (Индекс документа: 4911)
            > Интернет-сообщества обнажают потребности общества во внутренних связях, и виртуальные **сети** представляют материал для диагностики реальных **социальных** связей в обществе. Любая **социальная** **сеть** это прежде всего объем информации, которым один человек готов поделиться с другим. Однако **социальные** **сети** предполагают и определенную меру публичности, это своеобразный формат «Я и другие». Интернет-сообщество это мир идеальных людей, в котором они представляются такими, какими хотят, ...

3. (Индекс документа: 5678)
            > Статья посвящена исследованию феномена «социальная сеть» в русской и английской лингвокультуре. В этой связи между компонентами словосочетаний «социальная сеть» / «social network» устанавливается семантическая связь. Общими и для русского, и для английского языка являются лексемы, отражающие систему, организацию, контроль, коммуникацию, информацию, эмоции и ментальную сферу. В английской лингвокультуре наблюдается противоречивое отношение к **социальной** **сети:** с одной стороны, выявлены лекс...

4. (Индекс документа: 10207)
            > В статье дан обзор основных результатов изучения как реальных **социальных** **сетей** **(сетей** сотрудничества ученых и актеров, **сетей** цитирования научных публикаций, **сетей** друзей
и знакомых и т. д.), так и современных онлайновых **социальных** **сетей** (Twitter, Facebook и т. д.)
с точки зрения теории сложных **сетей.** На основе собственных исследований авторами выявлены
особенности восприятия некоторых сложных **сетей.**...

5. (Индекс документа: 5696)
            > Статья посвящена исследованию лексического аспекта аннотаций **социальных** **сетей** в английском и русском языках. В этой связи в аннотациях **социальных** **сетей** выявляются лексемы, отражающие отношение к данному явлению в английской и русской лингвокультуре. Несмотря на то, что и в английском, и в русском языке прослеживается положительная оценка **социальных** **сетей,** в ряде случаев выявлено отрицательное отношение, которое выражается в употреблении лексем, отражающих ограничение дост...

* Как видно из подсветки, TF-IDF успешно находит документы, содержащие точные ключевые слова из запроса, такие как 'модернизация' и 'производственных'. Ранжирование происходит на основе частоты и редкости этих слов.
* Однако этот подход полностью игнорирует семантику. Если в запросе указано 'модернизация производства', а в тексте — 'оптимизация технологических процессов на заводе', TF-IDF, скорее всего, не найдет этот документ или даст ему низкий ранг. Эта проблема будет решена с помощью семантического поиска на основе эмбеддингов.